In [12]:
from transformers import WhisperForConditionalGeneration, WhisperModel, WhisperProcessor, WhisperTokenizer
from datasets import load_dataset
import evaluate
import torch

In [4]:
transcribe_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")

Downloading: 100%|██████████| 1.98k/1.98k [00:00<00:00, 3.86MB/s]
d:\Users\shast\envs\whisper_env\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shast\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 3.06G/3.06G [06:57<00:00, 7.32MB/s]   


In [5]:
# Processor is the mixture of the FeatrueExtractor and tokenizer for the whisper model
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")

Downloading: 100%|██████████| 185k/185k [00:00<00:00, 213kB/s]  
Downloading: 100%|██████████| 830/830 [00:00<00:00, 700kB/s]
Downloading: 100%|██████████| 1.04M/1.04M [00:02<00:00, 347kB/s]
Downloading: 100%|██████████| 494k/494k [00:01<00:00, 267kB/s]  
Downloading: 100%|██████████| 52.7k/52.7k [00:00<00:00, 103kB/s] 
Downloading: 100%|██████████| 2.11k/2.11k [00:00<00:00, 224kB/s]
Downloading: 100%|██████████| 2.06k/2.06k [00:00<?, ?B/s]


In [10]:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
print(ds[0])

Found cached dataset librispeech_asr_dummy (C:/Users/shast/.cache/huggingface/datasets/hf-internal-testing___librispeech_asr_dummy/clean/2.1.0/d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b)


{'file': 'C:/Users/shast/.cache/huggingface/datasets/downloads/extracted/f479719f1bfb9350a98b166fb01ab5165fb756f54dc5c62e15bf9eb6d555a0bc/dev_clean/1272/128104/1272-128104-0000.flac', 'audio': {'path': 'C:/Users/shast/.cache/huggingface/datasets/downloads/extracted/f479719f1bfb9350a98b166fb01ab5165fb756f54dc5c62e15bf9eb6d555a0bc/dev_clean/1272/128104/1272-128104-0000.flac', 'array': array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ], dtype=float32), 'sampling_rate': 16000}, 'text': 'MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL', 'speaker_id': 1272, 'chapter_id': 128104, 'id': '1272-128104-0000'}


In [24]:
input_features = processor(ds[0]["audio"]["array"], return_tensors="pt", sampling_rate=16000).input_features
print(input_features.shape)
print(ds[0]["audio"]["array"].shape)

torch.Size([1, 80, 3000])
(93680,)


In [29]:
logits = transcribe_model.generate(input_features)


d:\Users\shast\envs\whisper_env\lib\site-packages\transformers\generation_utils.py:1296: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [30]:
print(logits.shape)
# predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(logits)

torch.Size([1, 26])


In [32]:
print(transcription)
print(ds[0]["text"])

['<|startoftranscript|><|en|><|transcribe|><|notimestamps|> Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.<|endoftext|>']
MISTER QUILTER IS THE APOSTLE OF THE MIDDLE CLASSES AND WE ARE GLAD TO WELCOME HIS GOSPEL
